In [27]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split

In [28]:
mnist = fetch_openml('mnist_784', version=1)

In [29]:
X, y = mnist.data.values, mnist.target.values

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=60000, random_state=42)
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=10000, random_state=42)
# take a sample of 10000 points from X_train and y_train

In [103]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
# create a clf rbf kernel
scaler = StandardScaler()
X_train_sample_scaled = scaler.fit_transform(X_train_sample)
clf = SVC()
param_grid = [{
    "kernel": ["rbf"],
    "C": [5, 6, 7],
    "gamma": [0.001, 0.0008, 0.0012]
}]


grid_search = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train_sample_scaled, y_train_sample)

GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [5, 6, 7], 'gamma': [0.001, 0.0008, 0.0012],
                          'kernel': ['rbf']}],
             return_train_score=True, scoring='accuracy')

After running the grid search multiple times, I noticed that gamma that always gave the best results is 0.001, so I removed the other values, that were 0.1, 0.01, and 1, to improve the performance of the grid search.

After testing various different 'C' I was able to find the best possible combination which resulted in a mean test score of 0.9429 on a sample of 10000 points from the training set.

In [104]:
# printing all results
cvres = grid_search.cv_results_
df = pd.DataFrame(cvres)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,50.055558,0.291148,27.029602,0.928448,5,0.001,rbf,"{'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}",0.944511,0.940294,0.941494,0.9421,0.001774,9,0.996550,0.99625,0.99580,0.99620,0.000308
1,45.401760,2.041783,26.073712,0.321180,5,0.0008,rbf,"{'C': 5, 'gamma': 0.0008, 'kernel': 'rbf'}",0.946611,0.941494,0.942394,0.9435,0.002230,1,0.993999,0.99295,0.99310,0.99335,0.000463
2,52.605706,1.277476,26.845855,0.704313,5,0.0012,rbf,"{'C': 5, 'gamma': 0.0012, 'kernel': 'rbf'}",0.946911,0.940594,0.940294,0.9426,0.003051,7,0.997600,0.99805,0.99730,0.99765,0.000308
3,47.686180,1.640839,25.986444,0.543750,6,0.001,rbf,"{'C': 6, 'gamma': 0.001, 'kernel': 'rbf'}",0.946011,0.941494,0.941194,0.9429,0.002203,4,0.997450,0.99775,0.99700,0.99740,0.000308
4,44.285718,1.810983,26.288811,0.449135,6,0.0008,rbf,"{'C': 6, 'gamma': 0.0008, 'kernel': 'rbf'}",0.945111,0.941794,0.942694,0.9432,0.001400,3,0.994899,0.99460,0.99445,0.99465,0.000187
5,42.583544,5.940133,21.390993,3.724340,6,0.0012,rbf,"{'C': 6, 'gamma': 0.0012, 'kernel': 'rbf'}",0.947211,0.940894,0.939994,0.9427,0.003211,5,0.998500,0.99865,0.99820,0.99845,0.000187
6,34.811302,0.565523,17.168096,0.471076,7,0.001,rbf,"{'C': 7, 'gamma': 0.001, 'kernel': 'rbf'}",0.945411,0.940294,0.941494,0.9424,0.002185,8,0.997900,0.99820,0.99760,0.99790,0.000245
7,31.653164,0.756594,17.011204,0.408026,7,0.0008,rbf,"{'C': 7, 'gamma': 0.0008, 'kernel': 'rbf'}",0.946611,0.942394,0.941194,0.9434,0.002323,2,0.996250,0.99625,0.99580,0.99610,0.000212
8,35.635822,1.362500,18.314353,0.426017,7,0.0012,rbf,"{'C': 7, 'gamma': 0.0012, 'kernel': 'rbf'}",0.946311,0.941194,0.940294,0.9426,0.002650,6,0.999250,0.99925,0.99850,0.99900,0.000354


In [106]:
# print the best parameters (CURRENT BEST C = 6, GAMMA =0.001, acc on test data = 0.9457)
print(grid_search.best_params_)

{'C': 5, 'gamma': 0.0008, 'kernel': 'rbf'}


In [107]:
clf_best = SVC(C=5, gamma=0.001)
clf_best.fit(X_train_sample_scaled, y_train_sample)
test_score = clf_best.score(scaler.transform(X_test), y_test)
print("Test Accuracy: {:.2f}%".format(test_score * 100))

Test Accuracy: 94.56%
